In [1]:
#using alpha_vantage because quandl gives only dattsa till March 2018

from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, date
import requests
import pandas as pd
import time
import quandl
import pandas as pd
import sys
import math
from alpha_vantage.techindicators import TechIndicators
from datapackage import Package
import urllib.request, json
from pymemcache.client import base

## Changed

In [2]:
support1 = 0
support2 = 0
resistance1 = 0
resistance2 = 0

In [3]:
def all_time_rsi_values(stock_para):
    ts = TechIndicators(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
    data, meta_data = ts.get_rsi(symbol=stock_para)
    return data['RSI'][-1]

In [4]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

def get_earnings_date():
    start_date = date.today()
    end_date = date.today()+timedelta(5)

    list_4_today=list()
    for single_date in daterange(start_date, end_date):
        time.sleep(5)
        x=single_date.strftime("%Y%m%d")
        urlto_search="https://api.earningscalendar.net/?date="+x
        with urllib.request.urlopen(urlto_search) as url:
            data = json.loads(url.read().decode())
            for dat in data:
                list_4_today.append(dat["ticker"])
    return list_4_today


In [5]:
def get_all_data(symbol_para):
    ts = TimeSeries(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
    try:
        data, meta_data = ts.get_daily_adjusted(symbol=symbol_para, outputsize='full')
    except:
        return pd.DataFrame()
    myData = data.rename(index=str, columns={"2. high": "High", "3. low": "Low","4. close":"Close"})
    myData['20d'] = np.round(myData.Close.rolling(window =20, center = False).mean(),2)
    myData['50d'] = np.round(myData.Close.rolling(window =50, center = False).mean(),2)
    myData['200d'] = np.round(myData.Close.rolling(window =200, center = False).mean(),2)
    return myData

In [6]:
#SMA rating
def SMA_rating(data_simple):
#     start_date = date(1998,1,2)
#     yesterday = (datetime.now() - timedelta(days=1))
#     end_date = date(yesterday.year,yesterday.month,yesterday.day)
    sma_rating = 0
    index = len(data_simple.index)-1
    if data_simple['Close'][index] > data_simple['50d'][index] > data_simple['200d'][index]:
        sma_rating=1
    elif data_simple['50d'][index] > data_simple['Close'][index]> data_simple['200d'][index]:
        sma_rating=2
    elif data_simple['50d'][index] > data_simple['200d'][index] > data_simple['Close'][index]:
        sma_rating=3
    elif data_simple['200d'][index] > data_simple['50d'][index] > data_simple['Close'][index]:
        sma_rating=4
    elif data_simple['200d'][index]> data_simple['Close'][index] > data_simple['50d'][index]:
        sma_rating=5
    elif data_simple['Close'][index] > data_simple['200d'][index] > data_simple['50d'][index]:
        sma_rating=6
    elif data_simple['Close'][index] > data_simple['50d'][index] > data_simple['200d'][index]:
        sma_rating=7
    return sma_rating

In [7]:
def MACD_rating(data_simple):
        data_simple['26 ema']=data_simple.Close.ewm(span=26).mean()
        data_simple['12 ema']=data_simple.Close.ewm(span=12).mean()
        data_simple['MACD'] = (data_simple['12 ema'] - data_simple['26 ema'])
        data_simple['signal_line']=data_simple.MACD.ewm(span=9).mean()
        data_simple['hist']=(data_simple['MACD']-data_simple['signal_line'])
        
        #MACD rating
        index = len(data_simple)-1
        slope = 0
        macd_rating = 0
        
        print("Current:",data_simple['MACD'][index])
        print("Previous day",data_simple['MACD'][index-1])
        
        #Condition for the slope
        if((data_simple['MACD'][index] - data_simple['MACD'][index-1])>0):
            slope = 1
        elif((data_simple['MACD'][index] - data_simple['MACD'][index-1])<0):
            slope = -1
            
        #Condition for the MACD rating
        if data_simple['MACD'][index] > data_simple['signal_line'][index] and data_simple['signal_line'][index] > 0:
            macd_rating=1
        elif data_simple['signal_line'][index] > data_simple['MACD'][index] and data_simple['MACD'][index] > 0:
            macd_rating=2
        elif data_simple['MACD'][index] < 0 and 0 < data_simple['signal_line'][index]:
            macd_rating=3
        elif data_simple['MACD'][index] < data_simple['signal_line'][index] and data_simple['signal_line'][index]< 0:
            macd_rating=4
        elif data_simple['signal_line'][index] < data_simple['MACD'][index] and data_simple['MACD'][index] < 0:
            macd_rating=5
        elif data_simple['MACD'][index] > 0 and 0 > data_simple['signal_line'][index]:
            macd_rating=6
        return macd_rating,slope

In [8]:
def daily_rsi_values(stock_para):
    ts = TechIndicators(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
    try:
        data, meta_data = ts.get_rsi(symbol=stock_para,interval='1min')
    except:
        return -1
    if(data.empty):
        return -1
    if(data['RSI'][-1]>70 or data['RSI'][-1]<30):
        ret_val = -1
    else:
        ret_val = 1
    return ret_val

## Changed

In [9]:
## Global Declarations

reversal = 3 #reversal amount set to 3 as default

# (price range, box size)
#function will provide box sizes according to traditional method
box_ranges = [(.25,.0625),
              (1,.125), 
              (5,.25),
              (20,.5),
              (100,1),
              (200,2),
              (500,4),
              (1000,5),
              (25000,50),
              (sys.maxsize,500)]

#to check and update box size according to current value of stock
def updateBoxSize(price):
    for i in range (len(box_ranges)):
        if price < box_ranges[i][0]:
            return box_ranges[i][1]
    return None


#using alpha_vantage because quandl gives only data till March 2018

#if current trend is x update x column appropriately or move to o column if needed
def updateX(item, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes):
    box_size = updateBoxSize(list1[-1])
   # print(math.floor(item["High"]))
    if ( math.floor(item["High"]) >= list1[-1]+box_size ):
        list1[-1] = math.floor(item["High"])
    #   print("Updated the x value to:"+str(list1[-1]))
        numberofXBoxes += 1
    elif ( math.ceil(item["Low"]) <= list1[-1]-reversal*box_size):
        list2.append(math.ceil(item["Low"]))
    #   print("Updated the o value to:"+str(list2[-1]))
        current_trend = 'o'
        numberofOBoxes += 1
    return current_trend

#if current trend is o update o column appropriately or move to x column if needed
def updateO(item, list1, list2, box_size, reversal, current_trend,  numberofXBoxes, numberofOBoxes):
    box_size = updateBoxSize(list2[-1])
    if ( math.ceil(item["Low"]) <= list2[-1]-box_size ):
        list2[-1] = math.ceil(item["Low"])
#        print("Updated the o value to:"+str(list2[-1]))
        numberofOBoxes += 1
    elif ( math.floor(item["High"]) >= list2[-1]+reversal*box_size):
        list1.append(math.floor(item["High"]))
 #       print("Updated the x value to:"+str(list1[-1]))
        current_trend = 'x'
        numberofXBoxes += 1
    return current_trend

#create the point and figure from scratch

def pointAndFigureCreate(box_size,current_trend, list1, list2, stockhilowdata, numberofXBoxes, numberofOBoxes):
    #iterate over each date for the stock and create x or o columns
    for index,row in stockhilowdata.iloc[0:].iterrows():
        if current_trend == 'x':
        #   print("The price is:"+str(row["High"]))
            current_trend = updateX(row, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes)
        elif current_trend=='o':
            current_trend = updateO(row, list1, list2, box_size, reversal, current_trend, numberofXBoxes, numberofOBoxes)
          #  print("The price is:"+str(row["Low"]))
    return current_trend

#check for a continous triple top pattern
def continous_triple_top(current_trend, list1):
    if(len(list1)<3):
        return -1
    ret_val = 0
    if current_trend == 'x':
        if((list1[-1]> list1[-2]) and (list1[-2] == list1[-3])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val
        
#check for a continous triple bottom pattern
def continous_triple_bottom(current_trend, list2):
    if(len(list2)<3):
        return -1
    ret_val = 0
    if current_trend == 'o':
        if((list2[-1] < list2[-2]) and (list2[-2] == list2[-3])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

#check for a continous double top pattern
def continous_double_top(current_trend, list1):
    if(len(list1)<2):
        return -1
    ret_val = 0
    if current_trend=='x':
        if(list1[-1]> list1[-2]):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

#check for a continous double bottom pattern
def continous_double_bottom(current_trend, list2):
    if(len(list2)<2):
        return -1
    ret_val = 0
    if current_trend == 'o':
        if(list2[-1]< list2[-2]):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def continous_quadruple_top(current_trend, list1):
    if(len(list1)<4):
        return -1
    ret_val = 0
    if current_trend == 'x' and len(list1)>3:
        if((list1[-1] > list1[-2]) and (list1[-2] == list1[-3] == list1[-4])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def continous_quadruple_bottom(current_trend, list2):
    if(len(list2)<4):
        return -1
    ret_val = 0
    if current_trend == 'o' and len(list2)>3:
        if((list2[-1] < list2[-2]) and (list2[-2] == list2[-3] == list2[-4])):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

import itertools
def spread_triple_top(current_trend, list1):
    ret_val = -1
    if current_trend == 'x' and len(list1)>7:
        iterprev = 0
        resistance = 0
        notstt = True
        for iter in list1[-2:-7:1]:
            if iterprev == iter:
                resistance = iter
                notstt = False
            if resistance!=0 and iter > resistance:
                notstt = True
                break
            iterprev = iter
        if(notstt == False):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def spread_triple_bottom(current_trend, list2):
    ret_val = -1
    if current_trend == 'o' and len(list2)>7:
        iterprev = 0
        support = 0
        notstb = True
        for iter in list2[-2:-7:1]:
            if iterprev == iter:
                support = iter
                notstb = False
            if support!=0 and iter<resistance:
                notstb = True
                break
            iterprev = iter
        if(notstb == False):
            ret_val = 1
        else:
            ret_val = -1
    return ret_val

def get_support_levels(list2):
    if(len(list2)>=2):
        return list2[-1],list2[-2]
    elif (len(list2)==1):
        return list2[-1],0
    else:
        return 0,0

def get_resistance_levels(list1):
    if(len(list1)>=2):
        return list1[-1],list1[-2]
    elif (len(list1)==1):
        return list1[-1],0
    else:
        return 0,0
    
    
def PnFMain(myData):
    #list1 stores the highest value of each x column by index
    #list2 stores the lowest value of each o column by index
    list1 = []
    list2 = []

    #current_trend stores the value of the most recent trend. we start off with x as default
    current_trend = 'x'

    numberofXBoxes = 1 # to get number of x boxes in all
    numberofOBoxes = 0 # to get number of o boxes in all

    #set box size according to price on day 1
    box_size = updateBoxSize(myData["High"].iloc[0])
    #append price on day 1 to list1 as we start with x
    list1.append(math.floor(myData["High"].iloc[0]))
    current_trend = pointAndFigureCreate(box_size, current_trend, list1, list2, myData, numberofXBoxes, numberofOBoxes)
    ret_val_triple_top = continous_triple_top(current_trend,list1)
    ret_val_triple_bottom = continous_triple_bottom(current_trend,list2)
    ret_val_double_top = continous_double_top(current_trend,list1)
    ret_val_double_bottom = continous_double_bottom(current_trend,list2)
    ret_val_quadruple_top = continous_quadruple_top(current_trend,list1)
    ret_val_quadruple_bottom = continous_quadruple_bottom(current_trend,list2)
    ret_val_spread_triple_top = spread_triple_top(current_trend,list1)
    ret_val_spread_triple_bottom = spread_triple_bottom(current_trend,list2)
    
    global support1
    global support2
    global resistance1
    global resistance2
    
    support1,support2 = get_support_levels(list2)
    resistance1,resistance2 = get_resistance_levels(list1)
    
    if ret_val_quadruple_top == 1:
        return 'a'
    elif ret_val_triple_top == 1:
        return 'b'
    elif ret_val_double_top == 1:
        return 'c'
    elif ret_val_quadruple_bottom == 1:
        return 'd'
    elif ret_val_triple_bottom == 1:
        return 'e'
    elif ret_val_double_bottom == 1:
        return 'f'
    else:
        return None
    

In [10]:
financial_list=list()
energy_list=list()
industrial_list=list()
comm_services_list=list()
materials_list=list()
health_list=list()
consumer_list=list()
technology_list=list()
consumer_stap_list=list()

sector_etf_list=['XLF','XLE','XLI','XLC','XLB','XLV','XLY','XLK','XLP']
sector_sma_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_macd_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_macd_slope={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_pnf_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_rsi_rating={'XLF':0,'XLE':0,'XLI':0,'XLC':0,'XLB':0,'XLV':0,'XLY':0,'XLK':0,'XLP':0}
sector_etf_to_names={'XLF':financial_list,'XLE':energy_list,'XLI':industrial_list,'XLC':comm_services_list,'XLB':materials_list,'XLV':health_list,'XLY':consumer_list,'XLK':technology_list,'XLP':consumer_stap_list}


In [11]:
package = Package('https://datahub.io/core/s-and-p-500-companies/datapackage.json')

In [12]:
for resource in package.resources:
    if resource.descriptor['datahub']['type'] == 'derived/csv':
        my_list=resource.read()
df = pd.DataFrame(my_list, columns=['Symbol','Name','Sector'])

In [13]:
for i in range(0,len(df)):
    if df['Sector'][i]=='Industrials':
        industrial_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Health Care':
        health_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Energy':
        energy_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Financials':
        financial_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Information Technology':
        technology_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Consumer Discretionary':
        consumer_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Materials':
        materials_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Telecommunication Services':
        comm_services_list.append(df['Symbol'][i])
    elif df['Sector'][i]=='Consumer Staples':
        consumer_stap_list.append(df['Symbol'][i])

In [14]:
df_rated_now=pd.DataFrame()

In [15]:
for sector in sector_etf_list:
#     if sector_sma_rating[sector]==1 and sector_macd_rating[sector]==1 and sector_macd_slope[sector]==1:
        for i in range(0,len(sector_etf_to_names[sector])):
            time.sleep(15)
            myData=get_all_data(sector_etf_to_names[sector][i])
            time.sleep(15)
            if(not myData.empty):
                macd_rating,slope = MACD_rating(myData)
                df_rated_now= df_rated_now.append({'Stock Symbol':sector_etf_to_names[sector][i], 'SMA':SMA_rating(myData), 'MACD':macd_rating,'MACD Slope':slope, 'PnF':PnFMain(myData), 'RSI':daily_rsi_values(sector_etf_to_names[sector][i]), 'Support1':support1, 'Support2':support2,'Resistance1':resistance1, 'Resistance2':resistance2 }, ignore_index=True)
            else:
                print('Data for stock not present in the API')
        print(df_rated_now)


Current: -2.5702172344478953
Previous day -2.801539922471335
Current: -1.1601347677328064
Previous day -1.2549566981768336
Current: -0.11961722189893464
Previous day -0.08158058635582677
Current: -1.4393983876604324
Previous day -1.5060296713094061
Current: -0.7185687202308202
Previous day -0.6605498146578341
Current: -3.7548009657146793
Previous day -4.174928979616084
Current: 0.2385401374230014
Previous day -0.05503951380404715
Current: -0.20087312457310702
Previous day -0.366700009575581
Current: 2.408296280177524
Previous day 2.4397146396548237
Current: -0.5026017987394198
Previous day -0.5781124755528673
Current: -0.6045129361085984
Previous day -0.7450404498701388
Current: -0.939599057741674
Previous day -1.296695350582155
Data for stock not present in the API
Current: -0.5889395302200207
Previous day -0.42175669530114845
Current: -0.4770178725916949
Previous day -0.540343738762715
Current: -1.3345659423116274
Previous day -1.548042136054022
Current: 1.5109231472096525
Previous d

KeyboardInterrupt: 

In [16]:
df_rated_now.to_csv('Rated_Stocks_All.csv', index=False)

In [17]:
def get_final_list():
    final_list = []
    pnf_list = ['a','b','c']
    for index,row in df_rated_now.iloc[0:].iterrows():
        if row['MACD'] == 1 and row['MACD Slope'] == 1 and row['SMA'] == 1 and row['PnF'] in pnf_list and row['RSI'] == 1:
            row_data = {'Symbol':row['Stock Symbol'] , 'Score':row['PnF'], 'Support1':row['Support1'],'Support2':row['Support2'],'Resistance1':row['Resistance1'],'Resistance2':row['Resistance2']}
            final_list.append(row_data)
    final_df = pd.DataFrame(final_list)
    if not final_df.empty:
        final_df['SymbolKey'] = final_df['Symbol']
        final_df.set_index('SymbolKey', inplace = True)
    else:
        return pd.DataFrame()
    return final_df

In [18]:
def get_close_price(stock_list):
    close_price = list()
    for row in stock_list.iterrows():
        symbol = row[1]['Symbol']
        print("Symbol is:", symbol)
        ts = TimeSeries(key='N2C8EGBURO9OUD1E', output_format='pandas',indexing_type='date')
        try:
            data, meta_data = ts.get_daily_adjusted(symbol=symbol, outputsize='full')
            myData = data.rename(index=str, columns={"2. high": "High", "3. low": "Low","4. close":"Close"})
            close_price.append(myData['Close'].iloc[-1])
        except:
            close_price.append(-1)
        time.sleep(15)
    stock_list['Close Price']=close_price
    return stock_list

In [19]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [20]:
def warn_for_earnings():
    start_date = date.today()
    end_date = date.today()+timedelta(30)

    earnings_date_list=list()
    for single_date in daterange(start_date, end_date):
        time.sleep(5)
        x=single_date.strftime("%Y%m%d")
        urlto_search="https://api.earningscalendar.net/?date="+x
        with urllib.request.urlopen(urlto_search) as url:
            data = json.loads(url.read().decode())
            year=int(x[0:4])
            month=int(x[4:6])
            day=int(x[6:8])
            for dat in data:   
                d0 = date.today()
                d1 = date(year, month, day)
                delta = d1 - d0
                earnings_date_list.append([dat["ticker"],x,delta.days])
                
    new_df=pd.DataFrame(columns=['Symbol','EarningsDate','Days to Earnings'])
    for ea in earnings_date_list:
        new_df=new_df.append({'Symbol':ea[0],'EarningsDate':ea[1],'Days to Earnings':ea[2]},ignore_index=True)
#     new_df.to_csv('earnings_warnings2.csv', index=False)
    return new_df

In [21]:
df_top_five_global = pd.DataFrame()

In [24]:
import requests
import json
import csv
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime  
from datetime import timedelta


# Following code makes relevant tokens necessary to conduct TD ameritrade activities on behalf of my account
# Note refresh token is valid for 90 days. It is used to generate auth token which is only valid for 30 mins
def get_auth_token():
    refresh_token = 'KNFyYv+KGMJ+nKBpcwa4rtmncy8+yBJajF/Huc6zfFJhoc440S5jKMWbEk1P7y7LGbLXs+hICnPoOEPoU+aMa/fljcHtHs4yzynVECMaHTlIRmg8+eaLh9Vc+WEKz00WI5pi+vR28u4ab79tMl/SytQeLvyH9P0y6eT9snxhZQ5clTdF6O/0NQYUTs3KZiAZpGUvjWO7YPmuv+BM+WspLwgZsXiI6pYa8nJwpiDJ6uoicJQApUJrSoAFAdKwC6aBs+BF8M/81Wt6iEpsKTnknEhldLa4FcXmsEhHIbVQBDyIfNMt/UEmGd3lTyuG5MSWz1bkcjv6mKTCE8Ho838HnXPx7KeLl3eKOTNtBaK08wQF65d95iBGs3OrzKu4VITJIyk1VAK0SIQcION4SDJDLqSjABdVEuqyfdBe5yKqXFRijgF+7YPvoiCxNXR100MQuG4LYrgoVi/JHHvlaq7yrxEX7I4pUK6eaG0sXBCazdygpsxc8yaE3S0IjUayIh2GhptY3kAo3ZCmP7TnwsQiHPZhfkNtYcJmnWbAdI/4O1Gqk+mqnHIujHtpFPIFaWaZa4/EHk8/zTXzkqGcCWSmjjjcxMNyGm11jHLnYd6egEDLpxeYS+bkb7UYGLn0aNeIExtYq5IjZeP63XRbpflGtiBLNgGmLlJfZg0S4RABknkp9HIicMhRZgFAnqr71FdXP4aGgbh15sRc8WOSlXWoo980e5CS8on+LEpvu8FGQAlmQj0dTs6bjXa7WAOLxvxHIN+k1dnPFkKgJlIMLnMyIVJpY2kE/todzRqo4QAeUb/ualjJvaiiPsGv0cNhgvpCRUBrNjmNfv5gLdz/4KIQDBy3g3jdYrWzFCNH54IS9cnqdmBfi/6b5bs1AA6AhC3kAqxMIE8IXI8=212FD3x19z9sWBHDJACbC00B75E'
    auth_params = {'grant_type':'refresh_token', 'refresh_token':refresh_token, 'client_id':'MIC02PQTCV9GGVDGI8OTPTGQWESRT0FA@AMER.OAUTHAP'}
    auth_api_url = 'https://api.tdameritrade.com/v1/oauth2/token'
    return "Bearer "+ requests.post(auth_api_url, data=auth_params).json()['access_token']


# Following code will get the option chain data using the authorized token
# Parameters: call_or_put: 'CALL'/'PUT'; symbol: string with stock symbol from passed in stock list
def get_raw_option_data(call_or_put, symbol):
    url = 'https://api.tdameritrade.com/v1/marketdata/chains'
    strike_count = '500' #need to figure out how to get ALL or just use an arbitraliy huge num
    strategy = 'SINGLE'
    authorization_header = {'Authorization': get_auth_token()}
    pay = {'symbol':symbol,'strikeCount':strike_count , 'strategy':strategy}
    if call_or_put == "CALL":
        return [requests.get(url, params = pay, headers = authorization_header).json()['callExpDateMap'], requests.get(url, params = pay, headers = authorization_header).json()['underlyingPrice']]
    elif call_or_put == "PUT":
        return [requests.get(url, params = pay, headers = authorization_header).json()['putExpDateMap'], requests.get(url, params = pay, headers = authorization_header).json()['underlyingPrice']]



# Following code does the filtering of the options data based on our chosen parameters
def processing(symbol,supp1,supp2, params, call_or_put):
    data = get_raw_option_data(call_or_put, symbol)
    agg_data = {}
    date_keys = []
    for (k,v) in data[0].items():
        date_keys.append([k,list(v.keys())])
        agg_data[k] = v
        
    #If there is no data regarding some symbol in API then just return an empty dataframe
    if not date_keys:
        return pd.DataFrame()
        
    target_list = []
    stock_price = data[1]
    
    if call_or_put == "CALL":
        for i in range(0,len(date_keys)):
            for j in range(0, len(date_keys[i][1])):
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < params['days_min'] or agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] > params['days_max']: # Days to Expiration Requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] < params['bid_min']: # Min/Max Bid requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['totalVolume'] < params['min_vol']: # Min/Max Volume Requirement!
                    continue
                if (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price) / stock_price < params['%OTM_lim']: # Percent OTM Requirement!
                    continue
                if 1 - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'] < params['prob_OTM_lim']: # Prob OTM Requirement!
                    continue
                support = 0
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']<supp1 and agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']>supp2:
                    support = 1
                # construct a dictionary object characteristics we want and append to list
                target = {'stock': symbol,
                         'contract': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['symbol'],
                         'strike_price':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
                         'stock_price':stock_price,
                         'days_to_exp':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'],
                         'bid': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'],
                         'ask': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'],
                         'last': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['last'],
                         'prob_OTM': 1 - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'],
                         '% OTM': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price) / stock_price,
                         'cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'])/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'max_cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']+((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'prob_touch': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*2,
                         'annual_return': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'] * 365 * 100)/((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']) * (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'])),
                         'btwn_supports':support}
                target_list.append(target)
    elif call_or_put == "PUT":
        for i in range(0,len(date_keys)):
            for j in range(0, len(date_keys[i][1])):
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] < params['days_min'] or agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'] > params['days_max']: # Days to Expiration Requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] < params['bid_min']: # Min/Max Bid requirement!
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['totalVolume'] < params['min_vol']: # Min/Max Volume Requirement!
                    continue
                if (stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']) / stock_price < params['%OTM_lim']: # Percent OTM Requirement!
                    continue
                delt = 1 + agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']
                print("The value of delta is", delt)
                print("The otm limit is", params['prob_OTM_lim'])
                if delt < params['prob_OTM_lim']: # Prob OTM Requirement!      + or -???
                    print("The value of delta", delt)
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']) < params['cov_ret']: # Covered return Requirement! 
                    continue
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*-2 < params['prob_touch']: # Probability of touching parameter      + or - ???
                    continue
                support = 0
                if agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']<supp1 and agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']>supp2:
                    support = 1
#Commented code checks for 24% annual return. If required in future
#                 value = (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid']*365*100)/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice']*agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'])
#                 if value<15:
#                     continue
    
                # Construct a dictionary object characteristics we want and append to list
                target = {'stock':symbol,
                         'contract': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['symbol'],
                         'strike_price':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
                         'stock_price':stock_price,
                         'days_to_exp':agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration'],
                         'bid': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'],
                         'ask': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['ask'],
                         'last': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['last'],
                         'prob_OTM': 1 + agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta'], # + or - ???
                         '% OTM': stock_price/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'],
                         'cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']-((agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] - stock_price)/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'max_cov_ret': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['mark']+((stock_price - agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'])/stock_price)*(365/agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'prob_touch': agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['delta']*2,
                         'annual_return': (agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['bid'] * 365 * 100)/(agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['strikePrice'] * agg_data[date_keys[i][0]][date_keys[i][1][j]][0]['daysToExpiration']),
                         'btwn_supports': support}# + or - ???
                
#                 print("The value of accepted delta",target['prob_OTM'])
                target_list.append(target)
    df = pd.DataFrame(target_list)
    if(df.empty):
        print('Got empty')
        return df
    df = df.set_index('contract', drop = False)
    return df



# Following function returns price data for a stock which can be used to construct a P&F chart
def return_candle_data(symbol):
    candle_url = 'https://api.tdameritrade.com/v1/marketdata/'+symbol+'/pricehistory'
    candle_header = {'Authorization': get_auth_token()}
    # Vary the properties below depending on how often you want the data
    # Ideally, we want to set up a web socket to continuously stream this data?
    period_type = 'day'
    period = 2
    freq_type = 'minute'
    freq = 1
    candle_payload = {'periodType': period_type, 'period': period, 'frequencyType': freq_type, 'frequency': freq}
    raw_candle_data = requests.get(candle_url, params = candle_payload, headers = candle_header).json()
    df = pd.DataFrame(raw_candle_data['candles'])
    return df

# Get the relevant options after proving stock symbol and parameter list for filtering. 
def main():
    stock_list = get_final_list() # Need to populate this array with stock list we derive from technical indicators
    earnings_df = warn_for_earnings()
    print('Earnings List:',earnings_df)
    print('Stock List Before:', stock_list)
    
    stock_list.to_csv('Stock_List.csv')
    stock_list.to_csv(datetime.today().strftime('%Y-%m-%d')+'.csv')
    stock_list.sort_values(by=['Score'], ascending = False)    
    stock_list = get_close_price(stock_list)

    safe_put_parameters = {'days_min': 1, 'days_max': 60, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.10, 'prob_OTM_lim': 0.95, 'cov_ret': 0.01, 'prob_touch': 0.01}
    mod_put_parameters = {'days_min': 1, 'days_max': 60, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.05, 'prob_OTM_lim': 0.90, 'cov_ret': 0.01, 'prob_touch': 0.01}
    risky_put_parameters = {'days_min': 1, 'days_max': 60, 'bid_min':0.25, 'bid_max': 10000000, 'min_vol': 5, 'max_vol': 1000000000, '%OTM_lim': 0.0, 'prob_OTM_lim': 0.85, 'cov_ret': 0.01, 'prob_touch': 0.01}
    
    df_top_five_all =  pd.DataFrame()
    for row in stock_list.iterrows():
        item = row[1]['Symbol']
        supp1 = row[1]['Support1']
        supp2 = row[1]['Support2']
        df = processing(item,supp1,supp2, risky_put_parameters, "PUT")
        if(not df.empty):
            df_top_five = df.sort_values(by=['annual_return'], ascending=False)
            df_top_five = df_top_five.head(5)
            if df_top_five_all.empty:
                df_top_five_all = df_top_five
            else:
                df_top_five_all = df_top_five_all.append(df_top_five)
                
    if(not df_top_five_all.empty):
        df_top_five_all = df_top_five_all.sort_values(by=['btwn_supports','stock','annual_return'], ascending=[False,True,False])
        df_top_five_all['Days to Earnings'] = 'Not Near'
        for row in df_top_five_all.iterrows():
            if row[1]['stock'] in earnings_df['Symbol']:
                d_days=int(warn_pd.loc[earnings_df['Symbol'] == str(row['symbol'])]['Days to Earnings'])-row['days_to_exp'][i]   
                if d_days>0:
                    if d_days<10:
                        df_top_five_all.drop(row, axis=0)
                    else:
                        row['Days to Earnings']=d_days
        df_top_five_all.to_csv('ALL_Stocks_Risky.csv')
    else:
        print("No contracts available in the risky criteria")
    
    df_top_five_all = pd.DataFrame()
    for row in stock_list.iterrows():
        item = row[1]['Symbol']
        supp1 = row[1]['Support1']
        supp2 = row[1]['Support2']
        df = processing(item,supp1,supp2, mod_put_parameters, "PUT")
        if(not df.empty):
            print('Processing for stock:',item)
            df_top_five = df.sort_values(by=['annual_return'], ascending=False)
            df_top_five = df_top_five.head(5)
            if df_top_five_all.empty:
                df_top_five_all = df_top_five
            else:
                df_top_five_all = df_top_five_all.append(df_top_five)
    if(not df_top_five_all.empty):
        df_top_five_all = df_top_five_all.sort_values(by=['btwn_supports','stock','annual_return'], ascending=[False,True,False])
        df_top_five_all['Days to Earnings'] = 'Not Near'
        for row in df_top_five_all.iterrows():
            if row[1]['stock'] in earnings_df['Symbol']:
                d_days=int(warn_pd.loc[earnings_df['Symbol'] == str(row['symbol'])]['Days to Earnings'])-row['days_to_exp'][i]   
                if d_days>0:
                    if d_days<10:
                        df_top_five_all.drop(row, axis=0)
                    else:
                        row['Days to Earnings']=d_days

        df_top_five_all.to_csv('ALL_Stocks_Mod.csv')
    else:
        print("No contracts available in the moderate criteria")
        
    df_top_five_all = pd.DataFrame()
    for row in stock_list.iterrows():
        item = row[1]['Symbol']
        supp1 = row[1]['Support1']
        supp2 = row[1]['Support2']
        df = processing(item,supp1,supp2, safe_put_parameters, "PUT")
        if(not df.empty):
            print('Processing for stock:',item)
            df_top_five = df.sort_values(by=['annual_return'], ascending=False)
            df_top_five = df_top_five.head(5)
            if df_top_five_all.empty:
                df_top_five_all = df_top_five
            else:
                df_top_five_all = df_top_five_all.append(df_top_five)
    if(not df_top_five_all.empty):
        df_top_five_all = df_top_five_all.sort_values(by=['btwn_supports','stock','annual_return'], ascending=[False,True,False]) 

        df_top_five_all['Days to Earnings'] = 'Not Near'
        for row in df_top_five_all.iterrows():
            if row[1]['stock'] in earnings_df['Symbol']:
                d_days=int(warn_pd.loc[earnings_df['Symbol'] == str(row['symbol'])]['Days to Earnings'])-row['days_to_exp'][i]   
                if d_days>0:
                    if d_days<10:
                        df_top_five_all.drop(row, axis=0)
                    else:
                        row['Days to Earnings']=d_days
        df_top_five_all.to_csv('ALL_Stocks_Safe.csv')                 

        print('Check the new csv')
        print('Processing complete!')
    else:
        print("No contracts available in the safe criteria")
    
if __name__ == "__main__":
    main()


Earnings List:     Symbol EarningsDate Days to Earnings
0      SNP     20190902                0
1    UNICY     20190902                0
2    GLPEY     20190902                0
3    COVTY     20190902                0
4    MDOMF     20190902                0
5     SSKN     20190902                0
6     RMED     20190902                0
7    PQUEQ     20190902                0
8     COUP     20190903                1
9     EGAN     20190903                1
10     HQY     20190903                1
11    CONN     20190903                1
12     GSM     20190903                1
13   SSNLF     20190903                1
14     YRD     20190903                1
15   DNZOY     20190903                1
16   LGRVF     20190903                1
17   ASEKY     20190903                1
18    NSSC     20190903                1
19    TUFN     20190903                1
20    PRED     20190903                1
21    CSTL     20190903                1
22    GWGH     20190903                1
2

## Future Scope Functions

In [ ]:
def get_sector_ratings(etf,sector_sma_rating,sector_macd_rating,sector_macd_slope):
    
    ts = TimeSeries(key='ZSAE2CXUXOLE67NH', output_format='pandas',indexing_type='date')
    data, meta_data = ts.get_daily_adjusted(symbol=etf, outputsize='full')
    myData = data.rename(index=str, columns={"2. high": "High", "3. low": "Low","4. close":"Close"})
    #print(myData.head())
    myDataSimple=myData[['Close']].copy()
    myDataSimple['20d'] = np.round(myDataSimple.Close.rolling(window =20, center = False).mean(),2)
    myDataSimple['50d'] = np.round(myDataSimple.Close.rolling(window =50, center = False).mean(),2)
    myDataSimple['200d'] = np.round(myDataSimple.Close.rolling(window =200, center = False).mean(),2)
    sma_rating_today=SMA_rating(myDataSimple)
    macd_rating_today,slope=MACD_rating(myDataSimple)
    time.sleep(12)
    return sma_rating_today,macd_rating_today,slope   


for etf in sector_etf_list:
    #print(etf)
    sector_sma_rating[etf],sector_macd_rating[etf],sector_macd_slope[etf]=get_sector_ratings(etf,sector_sma_rating,sector_macd_rating,sector_macd_slope)
    

In [96]:
def get_intra_day_data():
    ts = TimeSeries(key='ZSAE2CXUXOLE67NH', output_format='pandas')
    intra_day_data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
    intra_day_data = intra_day_data.rename(index=str, columns={"2. high": "High", "3. low": "Low","4. close":"Close"})
    intra_day_data['20m'] = np.round(intra_day_data.Close.rolling(window =20, center = False).mean(),2)
    intra_day_data['50m'] = np.round(intra_day_data.Close.rolling(window =50, center = False).mean(),2)
    intra_day_data['200m'] = np.round(intra_day_data.Close.rolling(window =200, center = False).mean(),2)
    return intra_day_data

In [97]:
#SMA rating
def SMA_rating_intra_day(data_simple):
#     start_date = date(1998,1,2)
#     yesterday = (datetime.now() - timedelta(days=1))
#     end_date = date(yesterday.year,yesterday.month,yesterday.day)
    sma_rating = 0
    sma_ratings = []
    for index,val in enumerate(data_simple['Close']):
        if data_simple['Close'][index] > data_simple['50m'][index] > data_simple['200m'][index]:
            sma_rating=1
        elif data_simple['50m'][index] > data_simple['Close'][index]> data_simple['200m'][index]:
            sma_rating=2
        elif data_simple['50m'][index] > data_simple['200m'][index] > data_simple['Close'][index]:
            sma_rating=3
        elif data_simple['200m'][index] > data_simple['50m'][index] > data_simple['Close'][index]:
            sma_rating=4
        elif data_simple['200m'][index]> data_simple['Close'][index] > data_simple['50m'][index]:
            sma_rating=5
        elif data_simple['Close'][index] > data_simple['200m'][index] > data_simple['50m'][index]:
            sma_rating=6
        elif data_simple['Close'][index] > data_simple['50m'][index] > data_simple['200m'][index]:
            sma_rating=7
        sma_ratings.append(sma_rating)
    return sma_ratings

In [98]:
def MACD_rating_intra_day(data_simple):
        data_simple['26 ema']=data_simple.Close.ewm(span=26).mean()
        data_simple['12 ema']=data_simple.Close.ewm(span=12).mean()
        data_simple['9 ema']=data_simple.Close.ewm(span=9).mean()
        data_simple['MACD'] = (data_simple['12 ema'] - data_simple['26 ema'])
        data_simple['signal_line']=data_simple.MACD.ewm(span=9).mean()
        data_simple['hist']=(data_simple['MACD']-data_simple['signal_line'])
        #MACD rating
        index = len(data_simple.index)-1
        macd_rating = 0
        macd_ratings = []
        for index,val in enumerate(data_simple['Close']):
            if data_simple['MACD'][index] > data_simple['signal_line'][index] and data_simple['signal_line'][index] > 0:
                macd_rating=1
            elif data_simple['signal_line'][index] > data_simple['MACD'][index] and data_simple['MACD'][index] > 0:
                macd_rating=2
            elif data_simple['MACD'][index] < 0 and 0 < data_simple['signal_line'][index]:
                macd_rating=3
            elif data_simple['MACD'][index] < data_simple['signal_line'][index] and data_simple['signal_line'][index]< 0:
                macd_rating=4
            elif data_simple['signal_line'][index] < data_simple['MACD'][index] and data_simple['MACD'][index] < 0:
                macd_rating=5
            elif data_simple['MACD'][index] > 0 and 0 > data_simple['signal_line'][index]:
                macd_rating=6
            macd_ratings.append(macd_rating)
        return macd_ratings